In [15]:
import tensorflow as tf
import os
import keras
import numpy as np
import easygdf
import mne
import json
import pandas as pd
from scipy.interpolate import interp1d


Save data from gdf file to csv

In [2]:
def save_data_in_csv():
    filenames = ["A01T.gdf", "A01E.gdf", "A02T.gdf", "A02E.gdf","A03T.gdf","A03E.gdf","A04T.gdf","A04E.gdf","A05T.gdf","A05E.gdf","A06T.gdf","A06E.gdf","A07T.gdf","A07E.gdf","A08T.gdf","A08E.gdf","A09T.gdf","A09E.gdf"]
    labels = ["A01T", "A01E", "A02T", "A02E","A03T","A03E","A04T","A04E","A05T","A05E","A06T","A06E","A07T","A07E","A08T","A08E","A09T","A09E"]

    # Iterate over the filenames and labels
    for filename, label in zip(filenames, labels):
        # Read the raw data
        filename='../datasets/BCICIV_2a_gdf/'+filename
        raw = mne.io.read_raw_gdf(filename)
        
        # Convert to DataFrame
        df = raw.to_data_frame()
        
        # Save to CSV
        df.to_csv('excel/'+label + '.csv', index=False)

See all information in files

In [3]:
def get_all_info():
    filename = '../datasets/BCICIV_2a_gdf/A02T.gdf'
    raw = mne.io.read_raw_gdf(filename)

    # Print basic information about the dataset
    print(raw.info)

    # Print channel names
    print(raw.ch_names)

    # Print data shape
    print(raw.get_data().shape)

    # Print sampling frequency
    print(raw.info['sfreq'])

    # Print events
    events, event_id = mne.events_from_annotations(raw)
    print(events)

    # Print event IDs
    print(event_id)

Get events and positions from gdf files


In [4]:
def get_event_position():

    filenames = ["A01T.gdf", "A02T.gdf","A03T.gdf","A04T.gdf", "A01E.gdf", "A02E.gdf","A03E.gdf","A04E.gdf","A05T.gdf","A05E.gdf","A06T.gdf","A06E.gdf","A07T.gdf","A07E.gdf","A08T.gdf","A08E.gdf","A09T.gdf","A09E.gdf"]
    labels = ["A01T", "A02T","A03T","A04T", "A01E", "A02E","A03E","A04E","A05T","A05E","A06T","A06E","A07T","A07E","A08T","A08E","A09T","A09E"]


    events_with_labels_arrays = {}

    for filename, label in zip(filenames, labels):

        filename='../datasets/BCICIV_2a_gdf/'+filename
        raw = mne.io.read_raw_gdf(filename)

        # Access event information from the raw object
        events, event_id = mne.events_from_annotations(raw)

        #the list have the dictionary that map each events' id  to events' labels
        events_id_labels = list(event_id.keys())

        #create array for gather togther events' type and positions
        temp= np.zeros_like(events)


        for i in range(len(events)):
            temp[i,0]=events[i,0]
            id=events[i,2]
            temp[i,1]=events_id_labels[id-1] # be in dalil ke dar list event ha ma id 10 drim vali araye events_id_labels k az tarigh key dastresi peyda krdim yedone adadash paiin tare

        events_with_labels_arrays[label] = temp

    
    # Get the sample indices of the events
    #event_positions_samples = events[:, 0]

    # Get the corresponding time points from raw
    #event_positions_times = raw.times[event_positions_samples]    

    return events_with_labels_arrays


Save data in array

In [6]:
def save_data_in_array():

    filenames = ["A01T.gdf","A02T.gdf","A03T.gdf","A04T.gdf","A01E.gdf", "A02E.gdf","A03E.gdf","A04E.gdf","A05T.gdf","A05E.gdf","A06T.gdf","A06E.gdf","A07T.gdf","A07E.gdf","A08T.gdf","A08E.gdf","A09T.gdf","A09E.gdf"]
    labels = ["A01T", "A02T","A03T","A04T", "A01E", "A02E","A03E","A04E","A05T","A05E","A06T","A06E","A07T","A07E","A08T","A08E","A09T","A09E"]


    # Create a dictionary to store data arrays
    data_arrays = {}


    # Iterate over the filenames and labels
    for filename, label in zip(filenames, labels):
        # Read the raw data
        filename='../datasets/BCICIV_2a_gdf/'+filename
        raw = mne.io.read_raw_gdf(filename)
        
        # Convert to DataFrame
        df = raw.to_data_frame()
        
        # Store data array in the dictionary
        data_arrays[label] = df.values  # Store the numpy array representation of the DataFrame

    # # Access data array for a specific label
    # example_label = "A01T"
    # example_data_array = data_arrays[example_label]    

    return data_arrays

    

Create data array for feeding 

In [7]:

def combine_data_labels(data_arrays,events_with_labels_arrays):
    labels = ["A01T","A02T", "A03T","A04T","A05T","A06T","A07T","A08T","A09T"]

    combined=[]

    for label in labels:

        temp_data=data_arrays[label]
        temp_events=events_with_labels_arrays[label]
        for i in range(len(temp_events)):
        
            if temp_events[i,1]==769 or temp_events[i,1]==770 or temp_events[i,1]==771 or temp_events[i,1]==772 :

                ev=temp_events[i,1]
                position=temp_events[i,0]
                # nth_sample=[]
                # for j in range(position, position+1000):
                #     if j==position+999:
                #         for k in range(1,23):
                #             nth_sample.append(temp_data[j,k])

                #         nth_sample.append(ev)    
                #         break
                #     for k in range(1,23):
                #         nth_sample.append(temp_data[j,k])
                sample=temp_data[position:position+1000,1:23]

                combined.append((sample,ev))

    return combined          


Interpolation

In [14]:
def interpolation(X):

    # Create an empty array to store the interpolated data
    final_interpolated_data = np.zeros((X.shape[0], 1050, X.shape[2]))

    # Perform interpolation for each sample separately
    for sample_idx in range(X.shape[0]):
        # Get the EEG data for the current sample
        sample_data = X[sample_idx, :, :]

        # Create an empty array to store the interpolated sample data
        interpolated_sample_data = np.zeros((1050, X.shape[2]))

        # Perform interpolation for each channel separately
        for channel in range(X.shape[2]):
            # Get the EEG data for the current channel
            channel_data = sample_data[:, channel]

            # Create an interpolation function for the current channel
            interp_func = interp1d(np.arange(1000), channel_data, kind='linear')

            # Interpolate the data to increase the number of samples to 1050
            interpolated_channel_data = interp_func(np.linspace(0, 999, 1050))

            # Store the interpolated data for the current channel
            interpolated_sample_data[:, channel] = interpolated_channel_data

        # Assign the interpolated sample data to the final array
        final_interpolated_data[sample_idx, :, :] = interpolated_sample_data

    return final_interpolated_data    

Cropping

In [9]:

def cropping(interpolated_data,raw_y_train):
    # Initialize empty lists to store cropped segments and corresponding labels
    cropped_segments = []
    segment_labels = []

    # Iterate over each sample
    for sample_idx in range(interpolated_data.shape[0]):
        # Get the EEG data for the current sample
        sample_data = interpolated_data[sample_idx, :, :]
        # Get the labels for the current sample
        sample_labels = raw_y_train[sample_idx]
        # Calculate the number of segments
        num_segments = (sample_data.shape[0] - 1000) // 10 + 1
        # Iterate over each segment
        for segment_idx in range(num_segments):
            # Calculate start index of the segment
            start_idx = segment_idx * 10
            # Extract the segment
            segment = sample_data[start_idx:start_idx + 1000, :]
            # Append the segment to the list of cropped segments
            cropped_segments.append(segment)
            # Extract the label for the segment
            segment_label = sample_labels
            # Append the label to the list of segment labels
            segment_labels.append(segment_label)

    # Convert the lists to numpy arrays
    cropped_segments = np.array(cropped_segments)
    segment_labels = np.array(segment_labels)

    return cropped_segments,segment_labels


Changes events to 0-3 classes labes

In [10]:
def events_to_classes(array):
    for i in range(len(array)):
        if array[i]==769:
            array[i]=0
        elif array[i]==770:
            array[i]=1
        elif array[i]==771:
            array[i]=2    
        elif array[i]==772:
            array[i]=3

    return array

